In [1]:
import gym
import numpy
import numpy as np
from IPython.display import clear_output
import torch as T

from PPO import Agent

In [2]:
from enviroment_class import CustomEnv

env = CustomEnv(obstacle_turn = True,
                vizualaze     = False,
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.005,#0.005
                num_obs       = 5,
                num_enemy     = 1,
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 1000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])
_ = env.reset()


pygame 2.1.0 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/parkhomenko/PycharmProjects/RL/venv/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:
N = 20
n_epochs = 3
alpha = 0.0003  ##0.003

agent = Agent(input_conv_dims = (15,3),
              allias_state = 5,
              enemy_state = 5, 
              n_actions = env.action_space.n,
              alpha=alpha,
              cuda = 'cuda:0',
              n_epochs=n_epochs)

n_games = 5000

best_score = env.reward_range[0]
score_history = []

learn_iters = 0
avg_score = 0
n_steps = 0

/home/parkhomenko/PycharmProjects/RL/venv/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Main

In [ ]:
for i in range(n_games):
    observation = env.reset()
    
    done = False
    score = 0
    hx, cx = T.zeros(64).to(agent.actor.device), T.zeros(64).to(agent.actor.device)
    H, C = T.zeros(64).to(agent.actor.device), T.zeros(64).to(agent.actor.device)
    
    agent.memory.clear_memory()
    
    while not done:
        action, prob, val, hx, cx = agent.choose_action(observation, hx, cx)
        observation_, reward, done, info = env.step(action)
        n_steps += 1
        score += reward
        
        state = observation['img']
        a = observation['posRobot']
        e = observation['target']
        agent.remember(state, a, e, action, prob, val, reward, done)
        
        if n_steps % N == 0:
            agent.learn(H , C)
            learn_iters += 1
            H = hx.detach()
            C = cx.detach()
            
        observation = observation_
        
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])

    if avg_score > best_score:
        best_score = avg_score
        agent.save_models()
    
    if not bool(i%10):clear_output()
    
    print('episode ', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
          'time step', n_steps, 'learning_steps', learn_iters)

episode  0 score -265.0 avg score -265.0 time step 338 learning_steps 16
... saving models ...
episode  1 score -11.1 avg score -138.1 time step 468 learning_steps 23
... saving models ...
episode  2 score -4.5 avg score -93.5 time step 508 learning_steps 25
... saving models ...
episode  3 score 39.5 avg score -60.3 time step 559 learning_steps 27
... saving models ...
episode  4 score 16.5 avg score -44.9 time step 727 learning_steps 36
episode  5 score -194.5 avg score -69.9 time step 828 learning_steps 41
episode  6 score -58.8 avg score -68.3 time step 871 learning_steps 43
episode  7 score -96.7 avg score -71.8 time step 905 learning_steps 45


In [ ]:
import matplotlib.pyplot as plt

def plot_learning_curve(x, scores, filename, lines=None):
    fig=plt.figure()
    ax=fig.add_subplot(111, label="1")
    ax2=fig.add_subplot(111, label="2", frame_on=False)

    ax.set_xlabel("Num games", color="C0")
    ax.set_ylabel("Epsilon", color="C0")
    ax.tick_params(axis='x', colors="C0")
    ax.tick_params(axis='y', colors="C0")

    N = len(scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(scores[max(0, t-20):(t+1)])

    ax2.scatter(x, running_avg, color="C1")
    ax2.axes.get_xaxis().set_visible(False)
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Score', color="C1")
    ax2.yaxis.set_label_position('right')
    ax2.tick_params(axis='y', colors="C1")

    if lines is not None:
        for line in lines:
            plt.axvline(x=line)

    plt.savefig(filename)

In [ ]:
x = [i+1 for i in range(len(score_history))]
plot_learning_curve(x, score_history, filename = 'PPO.png')

In [ ]:
env = CustomEnv(obstacle_turn = True,
                vizualaze     = False,
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.005,#0.005
                num_obs       = 5,
                num_enemy     = 1,
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 1000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])
_ = env.reset()


env.get_statistic( model = agent, num_games = 1000)